In [1]:
import os
import numpy as np
import sounddevice as sd
import tempfile
import scipy.io.wavfile
import cv2
import torch
import io
from pydub import AudioSegment
from IPython.display import Audio as IPyAudio
from playsound import playsound
from faster_whisper import WhisperModel
from transformers import VitsModel, AutoTokenizer
from ultralytics import YOLO
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

# === CONFIG ===
SAMPLE_RATE = 16000
DURATION = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# ------------------------------ Configuração do Modelo ------------------------------
def model_ollama(model, temperature=0.1):
    """Retorna uma instância do modelo LLM especificado."""
    return ChatOllama(model=model.lower().replace(" ", ""), temperature=temperature)

def model_response(user_query, chat_history, model_name):
    """Gera uma resposta baseada na interação do usuário com o modelo de IA."""
    try:
        llm = model_ollama(model_name)

        system_prompt = """
            Você é um assistente inteligente para automação residencial.
            Utilize raciocínio estruturado seguindo o método cadeia de pensamento (chain-of-thought):

            1. Compreenda claramente o comando do usuário.
            2. Identifique qual ação deve ser executada (ligar dispositivo, informar status, alterar configuração).
            3. Se o usuário quiser conversar seja simples e direto nas respostas.
            4. Execute (simule) a ação solicitada.
            5. Confirme a execução ou forneça a informação solicitada ao usuário de forma clara e amigável.

            Sempre responda com linguagem simples e amigável, adequada para uma conversa casual dentro de casa.
            Não mencione sua análise ou interpretação ao usuário, apenas confirme diretamente a execução da ação.
        """

        prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}")
        ])

        chain = prompt_template | llm | StrOutputParser()

        return chain.invoke({
            "chat_history": chat_history,
            "input": user_query
        })

    except Exception:
        return "❌ Desculpe, não consegui entender ou realizar essa ação. Por favor, tente novamente."



In [3]:
modelos_disponiveis = ["Llama 3.2", "phi4", "gemma3:27b"]
escolha = 1  # Índice do modelo desejado

In [4]:
# === ÁUDIO PARA TEXTO ===
def capture_audio_and_transcribe():
    print("🎙️ Gravando áudio...")
    audio = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
    sd.wait()
    print("✅ Gravação concluída.")
    
    audio = np.squeeze(audio)
    audio = np.clip(audio, -1.0, 1.0)
    audio_int16 = (audio * 32767).astype(np.int16)
    
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        scipy.io.wavfile.write(tmp.name, SAMPLE_RATE, audio_int16)
        audio_path = tmp.name
    
    model = WhisperModel("medium", compute_type="float32", device=DEVICE)
    segments, info = model.transcribe(audio_path, language="pt")
    os.remove(audio_path)
    
    transcription = " ".join([seg.text for seg in segments])
    print(f"📝 Transcrição: {transcription}")
    return transcription

# === VISÃO POR CÂMERA ===
def capture_image_and_describe():
    model = YOLO("yolov8n.pt")
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return "Nenhuma imagem capturada."

    results = model(frame)
    names = results[0].names
    boxes = results[0].boxes

    objetos = []
    for box in boxes:
        cls_id = int(box.cls[0])
        label = names[cls_id]
        conf = float(box.conf[0])
        objetos.append((label, conf))

    if not objetos:
        return "Nenhum objeto detectado."

    descricoes = []
    for label, conf in objetos:
        conf_percent = int(conf * 100)
        if conf >= 0.7:
            descricoes.append(f"{label} ({conf_percent}%)")
        elif conf >= 0.4:
            descricoes.append(f"possível {label} ({conf_percent}%)")
        else:
            descricoes.append(f"{label} incerto ({conf_percent}%)")

    return f"Objetos detectados: {', '.join(descricoes)}."


# ------------------------------ Configuração do Modelo ------------------------------
def model_ollama(model, temperature=0.1):
    """Retorna uma instância do modelo LLM especificado."""
    return ChatOllama(model=model.lower().replace(" ", ""), temperature=temperature)

def ask_llm_ollama(user_query, chat_history, model_name="phi4"):
    """Gera uma resposta baseada na interação do usuário com o modelo de IA."""
    try:
        llm = model_ollama(model_name)

        # system_prompt = """
        #     Você é um assistente inteligente para automação residencial.
        #     Utilize raciocínio estruturado seguindo o método cadeia de pensamento (chain-of-thought):

        #     1. Compreenda claramente o comando do usuário.
        #     2. Identifique qual ação deve ser executada (ligar dispositivo, informar status, alterar configuração).
        #     3. Se o usuário quiser conversar seja simples e direto nas respostas.
        #     4. Execute (simule) a ação solicitada.
        #     5. Confirme a execução ou forneça a informação solicitada ao usuário de forma clara e amigável.

        #     Sempre responda com linguagem simples e amigável, adequada para uma conversa casual dentro de casa.
        #     Não mencione sua análise ou interpretação ao usuário, apenas confirme diretamente a execução da ação.
        # """
        system_prompt = """
            Você é um assistente doméstico com percepção visual e auditiva.

            - Use a descrição da câmera, que inclui objetos detectados e seus níveis de confiança.
            - Avalie se o que foi detectado é confiável ou não.
            - Baseie sua resposta na fala do usuário e nos objetos com maior confiança.
            - Ignore objetos com baixa confiança, ou mencione que são incertos.
            - Seja claro, educado e direto.

            Exemplos:
            - Se um objeto tem confiança > 80%, considere-o confirmado.
            - Se está entre 40% e 80%, mencione como "possível".
            - Abaixo disso, trate como incerto e evite afirmar.

            Fale sempre em português, como se estivesse presente no ambiente.
            """


        prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}")
        ])

        chain = prompt_template | llm | StrOutputParser()

        return chain.invoke({
            "chat_history": chat_history,
            "input": user_query
        })

    except Exception:
        return "❌ Desculpe, não consegui entender ou realizar essa ação. Por favor, tente novamente."

# === TEXTO PARA ÁUDIO ===
def speak_text_with_mms(text):
    from transformers import VitsModel, AutoTokenizer
    import torch
    import io
    import scipy.io.wavfile
    import numpy as np
    from IPython.display import Audio, display

    model = VitsModel.from_pretrained("facebook/mms-tts-por")
    tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-por")

    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs).waveform

    audio = output.squeeze().numpy()
    audio = audio / np.max(np.abs(audio))  # normaliza para [-1, 1]
    audio_int16 = (audio * 32767).astype(np.int16)

    # Cria buffer de áudio em memória (formato WAV)
    buffer = io.BytesIO()
    scipy.io.wavfile.write(buffer, rate=model.config.sampling_rate, data=audio_int16)
    buffer.seek(0)

    # Exibe e toca o áudio automaticamente no notebook
    display(Audio(buffer.read(), rate=model.config.sampling_rate, autoplay=True))



In [5]:
history = []
while True:
    user_input = capture_audio_and_transcribe()
    if not user_input.strip():
        continue
    if "sair" in user_input.lower():
        print("👋 Encerrando assistente.")
        break

    vision_desc = capture_image_and_describe()
    full_prompt = f"{user_input}\nVisão: {vision_desc}"
    history.append(HumanMessage(content=full_prompt))
    modelo_escolhido = modelos_disponiveis[escolha - 1] if 0 < escolha <= len(modelos_disponiveis) else "phi4"
    print(f"\n✅ Modelo selecionado: {modelo_escolhido}")
    llm_response = model_response(full_prompt, history, modelo_escolhido)
    print("🤖:", llm_response)
    speak_text_with_mms(llm_response)

    history.append(HumanMessage(content=user_input))
    history.append(AIMessage(content=llm_response))

🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição:  Quais objetos estava vendo aqui na minha mão?

0: 480x640 3 persons, 1 cell phone, 38.6ms
Speed: 2.0ms preprocess, 38.6ms inference, 64.4ms postprocess per image at shape (1, 3, 480, 640)

✅ Modelo selecionado: Llama 3.2
🤖: Parece que você está segurando um objeto que pode ser uma celular, mas não estou seguro. Vou tentar ajudar! Você quer saber se é uma celular ou algo mais?


🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição: 
🎙️ Gravando áudio...
✅ Gravação concluída.
📝 Transcrição:  Parece que você está segurando um objeto que pode ser uma celular.

0: 480x640 1 person, 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

✅ Modelo selecionado: Llama 3.2
🤖: Sim, parece que a confiança está aumentando! Você está certinho, é provável que seja uma celular. Posso tentar ajudar com algo? Você quer ligá-la ou fazer algo com ela?


KeyboardInterrupt: 